Sieć perceptronowa - nasz

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

np.random.seed(42)

# ---- WCZYTYWANIE I CZYSZCZENIE ----
data = pd.read_csv('XAU_1d_data.csv', delimiter=";")
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date') # Gwarancja chronologii

# Feature Engineering
data['Target'] = data['Close'].shift(-1)
data = data.dropna()

features = ['Open', 'High', 'Low', 'Volume', 'Close']
X_raw = data[features].values
y_raw = data['Target'].values.reshape(-1, 1)

# ---- PODZIAŁ CHRONOLOGICZNY (70%, 15%, 15%) ----
n = len(X_raw)
train_idx = int(n * 0.7)
val_idx = int(n * 0.85)

X_train_raw, y_train_raw = X_raw[:train_idx], y_raw[:train_idx]
X_val_raw, y_val_raw = X_raw[train_idx:val_idx], y_raw[train_idx:val_idx]
X_test_raw, y_test_raw = X_raw[val_idx:], y_raw[val_idx:]

# ---- SKALOWANIE ----
scaler_x = StandardScaler().fit(X_train_raw)
scaler_y = StandardScaler().fit(y_train_raw)

X_train, y_train = scaler_x.transform(X_train_raw), scaler_y.transform(y_train_raw)
X_val, y_val = scaler_x.transform(X_val_raw), scaler_y.transform(y_val_raw)
X_test, y_test = scaler_x.transform(X_test_raw), scaler_y.transform(y_test_raw)

print(f"Dane gotowe. Trening: {len(X_train)}, Walidacja: {len(X_val)}, Test: {len(X_test)}")

Rozmiary: Train: (3669, 5), Val: (786, 5), Test: (787, 5)


In [ ]:
class CustomMLPRegression:
    def __init__(self, layer_sizes):
        self.weights = []
        self.biases = []
        for i in range(len(layer_sizes) - 1):
            # Inicjalizacja He dla ReLU
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i])
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        # Buffory dla momentum
        self.v_w = [np.zeros_like(w) for w in self.weights]
        self.v_b = [np.zeros_like(b) for b in self.biases]

    def relu(self, x): return np.maximum(0, x)
    def relu_deriv(self, x): return (x > 0).astype(float)

    def forward(self, X):
        self.activations = [X]
        self.zs = []
        curr = X
        for i in range(len(self.weights)):
            z = np.dot(curr, self.weights[i]) + self.biases[i]
            self.zs.append(z)
            curr = self.relu(z) if i < len(self.weights) - 1 else z # Ostatnia warstwa liniowa dla regresji
            self.activations.append(curr)
        return self.activations[-1]

    def backward(self, y_true, lr, momentum):
        error = self.activations[-1] - y_true
        deltas = [error]
        
        # Propagacja błędu wstecz
        for i in range(len(self.weights) - 1, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.relu_deriv(self.zs[i-1])
            deltas.append(delta)
        deltas.reverse()

        # Aktualizacja wag z Momentum
        for i in range(len(self.weights)):
            grad_w = np.dot(self.activations[i].T, deltas[i]) / y_true.shape[0]
            grad_b = np.mean(deltas[i], axis=0, keepdims=True)
            
            self.v_w[i] = momentum * self.v_w[i] - lr * grad_w
            self.v_b[i] = momentum * self.v_b[i] - lr * grad_b
            
            self.weights[i] += self.v_w[i]
            self.biases[i] += self.v_b[i]

    def train(self, X, y, epochs, lr, momentum):
        for _ in range(epochs):
            self.forward(X)
            self.backward(y, lr, momentum)

Rozpoczynam trening MLP (Własna implementacja)...


In [ ]:
def get_metrics(model, X, y_scaled, scaler_y):
    pred_scaled = model.forward(X)
    p = scaler_y.inverse_transform(pred_scaled)
    t = scaler_y.inverse_transform(y_scaled)
    return mean_squared_error(t, p), mean_absolute_error(t, p), r2_score(t, p)

results_mlp = []
# Definicja 4 wartości dla każdego kluczowego parametru
params_to_test = {
    'lr': [0.001, 0.01, 0.05, 0.1],
    'epochs': [500, 1000, 1500, 2000],
    'layers': [[16], [16, 16], [32, 16], [32, 16, 8]], # liczba warstw/neuronów
    'momentum': [0.0, 0.5, 0.9, 0.95]
}

print("Rozpoczynam badanie parametrów własnego MLP...")

# Przykładowa pętla badania wpływu Learning Rate (LR)
for lr in params_to_test['lr']:
    for r in range(5): # 5 powtórzeń
        model = CustomMLPRegression([X_train.shape[1], 16, 1]) # bazowa architektura
        model.train(X_train, y_train, epochs=1000, lr=lr, momentum=0.9)
        mse, mae, r2 = get_metrics(model, X_test, y_test, scaler_y)
        results_mlp.append({'param': 'lr', 'val': lr, 'run': r, 'mse': mse, 'r2': r2})

# UWAGA: Analogiczne pętle należy powtórzyć dla reszty słownika params_to_test
# (Dla czytelności wklejam schemat badania LR, w sprawozdaniu powtórz dla reszty)

df_mlp = pd.DataFrame(results_mlp)
df_mlp.to_excel('analiza_mlp_regresja.xlsx', index=False)

Pliki Excel zostały wygenerowane pomyślnie!


In [ ]:
def create_sequences(X, y, time_steps=7):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 7
X_train_seq, y_train_seq = create_sequences(X_train, y_train, TIME_STEPS)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, TIME_STEPS)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, TIME_STEPS)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Flatten, Input
from tensorflow.keras.optimizers import Adam, SGD

def run_keras_experiment(model_type='LSTM'):
    final_results = []
    
    # 4 wartości dla parametru "liczba warstw"
    layer_configs = [1, 2, 3, 4] 
    
    for layers in layer_configs:
        print(f"Testuję {model_type} z liczbą warstw: {layers}")
        for r in range(5):
            model = Sequential()
            model.add(Input(shape=(TIME_STEPS, X_train.shape[1])))
            
            # Dodawanie warstw zgodnie z parametrem
            for i in range(layers):
                if model_type == 'LSTM':
                    # return_sequences=True jest wymagane, gdy kolejna warstwa to też LSTM
                    model.add(LSTM(32, return_sequences=(i < layers - 1)))
                else: # CNN
                    model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
            
            if model_type == 'CNN': model.add(Flatten())
            model.add(Dense(1))
            
            model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
            model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, verbose=0)
            
            # Ewaluacja
            pred = model.predict(X_test_seq, verbose=0)
            p = scaler_y.inverse_transform(pred)
            t = scaler_y.inverse_transform(y_test_seq)
            
            final_results.append({
                'model': model_type,
                'param': 'layer_count',
                'val': layers,
                'run': r,
                'mse': mean_squared_error(t, p),
                'r2': r2_score(t, p)
            })
            
    return final_results

# Uruchomienie
results_lstm = run_keras_experiment('LSTM')
results_cnn = run_keras_experiment('CNN')

# Zapis do Excela (Podstawa do wykresów w sprawozdaniu)
pd.concat([pd.DataFrame(results_lstm), pd.DataFrame(results_cnn)]).to_excel('wyniki_zaawansowane.xlsx')

2026-01-07 10:52:16.175107: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-07 10:52:16.193389: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-07 10:52:16.805500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-07 10:52:18.652162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

--- Start eksperymentów dla LSTM ---
Testuję learning_rate = 0.0001... Done.
Testuję learning_rate = 0.001... Done.
Testuję learning_rate = 0.01... Done.
Testuję learning_rate = 0.05... Done.
Testuję epochs = 20... Done.
Testuję epochs = 50... Done.
Testuję epochs = 100... Done.
Testuję epochs = 150... Done.
Testuję units_filters = 16... Done.
Testuję units_filters = 32... Done.
Testuję units_filters = 64... Done.
Testuję units_filters = 128... Done.
Testuję batch_size = 16... Done.
Testuję batch_size = 32... Done.
Testuję batch_size = 64... Done.
Testuję batch_size = 128... Done.
--- Start eksperymentów dla CNN ---
Testuję learning_rate = 0.0001... Done.
Testuję learning_rate = 0.001... Done.
Testuję learning_rate = 0.01... Done.
Testuję learning_rate = 0.05... Done.
Testuję epochs = 20... Done.
Testuję epochs = 50... Done.
Testuję epochs = 100... Done.
Testuję epochs = 150... Done.
Testuję units_filters = 16... Done.
Testuję units_filters = 32... Done.
Testuję units_filters = 64... D